# GSW 가 40년만에 우승한 이유는 무엇인가.

### 분석 범위
- 우승 전(~2014) vs 우승 시즌(2015 ~ 2016)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
games = pd.read_csv("Data/games.csv", parse_dates=["GAME_DATE_EST"])
games_details = pd.read_csv("Data/games_details.csv", low_memory=False)
teams = pd.read_csv("Data/teams.csv")
ranking = pd.read_csv("Data/ranking.csv")
player = pd.read_csv("Data/players.csv")

## 가설
1. 3점슛 성공률이 높을 수록 승률이 높다.
2. 3점슛 시도 횟수가 높을수록 승률이 높을까?

In [ ]:
games_details.info()

In [ ]:
# 중복 PK 제거
games = games.drop_duplicates(subset=['GAME_ID'], keep='first')

# FG3A, FG3M을 int로 변환 (NaN은 0으로 처리)
games_details['FG3A'] = pd.to_numeric(games_details['FG3A'], errors='coerce').fillna(0).astype(int)
games_details['FG3M'] = pd.to_numeric(games_details['FG3M'], errors='coerce').fillna(0).astype(int)

games_details['COMMENT'] = games_details['COMMENT'].astype('string').fillna('')
games_details['MIN'] = games_details['MIN'].astype('string').fillna('00:00')
games_details['TEAM_ABBREVIATION'] = games_details['TEAM_ABBREVIATION'].astype('string').fillna('')
games_details['TEAM_CITY'] = games_details['TEAM_CITY'].astype('string').fillna('')
games_details['PLAYER_NAME'] = games_details['PLAYER_NAME'].astype('string').fillna('')
games_details['NICKNAME'] = games_details['NICKNAME'].astype('string').fillna('')
games_details['START_POSITION'] = games_details['START_POSITION'].astype('string').fillna('')

# Warriors 팀 ID 설정
gsw_team_id = 1610612744  # Golden State Warriors

In [ ]:
games_details.info()

In [ ]:
games.head()

In [ ]:
# Warriors 홈 경기 추출
gsw_home = games[games['TEAM_ID_home'] == gsw_team_id].copy()
gsw_home['WIN'] = gsw_home['HOME_TEAM_WINS'].astype(int)
gsw_home['TEAM_ID'] = gsw_team_id

# Warriors 원정 경기 추출
gsw_away = games[games['TEAM_ID_away'] == gsw_team_id].copy()
gsw_away['WIN'] = (1 - gsw_away['HOME_TEAM_WINS']).astype(int)
gsw_away['TEAM_ID'] = gsw_team_id

# 홈+원정 경기 합치기
gsw_all = pd.concat([gsw_home, gsw_away], ignore_index=True)

# 시즌별 승률 계산
season_stats = gsw_all.groupby('SEASON').agg({
    'WIN': ['sum', 'count', 'mean']
}).reset_index()

# 컬럼명 정리
season_stats.columns = ['SEASON', 'WINS', 'GAMES', 'WIN_PCT']

# 우승 시즌(2015) 전후 비교
print("\n📊 우승 전후 비교")
before_2015 = season_stats[season_stats['SEASON'] < 2015]['WIN_PCT'].mean()
season_2015 = season_stats[season_stats['SEASON'] == 2015]['WIN_PCT'].values[0] if 2015 in season_stats['SEASON'].values else None
after_2015 = season_stats[season_stats['SEASON'] > 2015]['WIN_PCT'].mean()

# 시즌별 승률 그래프
plt.figure(figsize=(12, 6))
plt.plot(season_stats['SEASON'], season_stats['WIN_PCT'], 
         marker='o', linewidth=2, markersize=8)

# 2015 우승 시즌 기점
if 2015 in season_stats['SEASON'].values:
    plt.axvline(x=2015, color='red', linestyle='--', 
                label='2015 우승 시즌', linewidth=2)

plt.xlabel('시즌', fontsize=12)
plt.ylabel('승률', fontsize=12)
plt.title('Golden State Warriors 시즌별 승률 추이', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

# NBA Warriors 승률 시각화
전체 시즌을 시각화 해 봤을 때, 2012년부터 2015년 사이의 승률이 크게 상승하고 2015년 후반, 2016년 초반에 최고 승률을 달성합니다.

In [ ]:

df_games_gsw = games[
    (games['TEAM_ID_home'] == gsw_team_id) | 
    (games['TEAM_ID_away'] == gsw_team_id)
].copy()

# Warriors가 홈/원정인지에 따라 컬럼 선택
df_games_gsw['FG3_PCT'] = df_games_gsw.apply(
    lambda row: float(row['FG3_PCT_home']) if row['TEAM_ID_home'] == gsw_team_id 
                else float(row['FG3_PCT_away']),
    axis=1
)

df_games_gsw['WIN'] = df_games_gsw.apply(
    lambda row: int(row['HOME_TEAM_WINS']) if row['TEAM_ID_home'] == gsw_team_id 
                else int(1 - row['HOME_TEAM_WINS']),
    axis=1
)

# 시즌별 통계
key_seasons = [2012, 2013, 2014, 2015, 2016]
season_stats = df_games_gsw[df_games_gsw['SEASON'].isin(key_seasons)].groupby('SEASON').agg({
    'FG3_PCT': 'mean',   # 평균 3점슛 성공률
    'WIN': 'mean'        # 승률
}).reset_index()

season_stats


In [ ]:
# 그래프 크기 설정
fig, ax1 = plt.subplots(figsize=(14, 7))

# 첫 번째 y축: 승률
color1 = '#1f77b4'
ax1.set_xlabel('시즌', fontsize=13, fontweight='bold')
ax1.set_ylabel('승률', fontsize=13, fontweight='bold', color=color1)
ax1.plot(season_stats['SEASON'], season_stats['WIN'], 
         marker='o', linewidth=3, markersize=10, 
         color=color1, label='승률')
ax1.tick_params(axis='y', labelcolor=color1, labelsize=11)
ax1.grid(True, alpha=0.2)

# 두 번째 y축: 3점슛 성공률
ax2 = ax1.twinx()
color2 = '#ff7f0e'
ax2.set_ylabel('3점슛 성공률', fontsize=13, fontweight='bold', color=color2)
ax2.plot(season_stats['SEASON'], season_stats['FG3_PCT'], 
         marker='s', linewidth=3, markersize=10, 
         color=color2, label='3점슛 성공률')
ax2.tick_params(axis='y', labelcolor=color2, labelsize=11)

# 2015 우승 시즌 강조
ax1.axvline(x=2015, color='red', linestyle=':', 
            linewidth=3, alpha=0.6, label='2015 우승')

# 제목
plt.title('Golden State Warriors: 3점슛 성공률과 승률의 관계 (2012-2016)', 
          fontsize=15, fontweight='bold', pad=20)

# 범례
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, 
           loc='upper left', fontsize=11, framealpha=0.9)

plt.tight_layout()
plt.show()

## 시즌 별 SW 3점슛 성공률과 승률의 관계
- 승률이 가장 높은 2015 ~ 2016년에 3점슛 성공률은 오히려 떨어짐
- 2012년에 3점슛 성공률이 가장 높았는데 승률은 최저

In [ ]:
plt.figure(figsize=(10, 8))

# 산점도
plt.scatter(season_stats['FG3_PCT'], season_stats['WIN'], 
            s=200, alpha=0.6, c=season_stats['SEASON'], 
            cmap='viridis', edgecolors='black', linewidth=2)

# 각 점에 시즌 라벨 추가
for idx, row in season_stats.iterrows():
    plt.annotate(f"{int(row['SEASON'])}", 
                 (row['FG3_PCT'], row['WIN']),
                 fontsize=11, fontweight='bold',
                 ha='center', va='center')

# 추세선 추가
z = np.polyfit(season_stats['FG3_PCT'], season_stats['WIN'], 1)
p = np.poly1d(z)
plt.plot(season_stats['FG3_PCT'], p(season_stats['FG3_PCT']), 
         "r--", alpha=0.8, linewidth=2, label='추세선')

plt.xlabel('3점슛 성공률', fontsize=13, fontweight='bold')
plt.ylabel('승률', fontsize=13, fontweight='bold')
plt.title('Golden State Warriors: 3점슛 성공률 vs 승률 상관관계', 
          fontsize=15, fontweight='bold')
plt.colorbar(label='시즌')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# 상관계수 출력
correlation = season_stats['FG3_PCT'].corr(season_stats['WIN'])
print(f"\n📊 상관계수: {correlation:.3f}")
if correlation > 0.7:
    print("→ 강한 양의 상관관계! 3점슛 성공률이 높을수록 승률도 높다! ✅")
elif correlation > 0.4:
    print("→ 중간 정도의 양의 상관관계")
else:
    print("→ 약한 상관관계")

## 시즌 별 GSW 3점슛 성공률 vs 승률 상관관계
- 추세선이 양의 기울기를 보이지만 데이터 분포가 산발적
- 2013년: 3점슛 잘 못던지고(37.5%), 승률도 낮음(59%)
- 2016년: 3점슛 엄청 잘 던지고(39%), 승률 최고(84%)
- 2012년: 3점슛 좀 던졌는데(40%), 승률은 낮음(57%)

명확한 상관관계가 없어 보인다.

2. 그럼 3점슛 시도 횟수가 높을수록 승률이 높을까?

In [ ]:
# Warriors 경기별 3점슛 시도 횟수 집계
warriors_details = games_details[games_details['TEAM_ID'] == gsw_team_id]
game_attempts = warriors_details.groupby('GAME_ID')['FG3A'].sum().reset_index()

# games 데이터와 병합
joined = games.merge(game_attempts, on='GAME_ID', how='inner')

# Warriors 승패 여부 계산
joined['WIN'] = joined.apply(
    lambda row: int(row['HOME_TEAM_WINS']) if row['TEAM_ID_home'] == gsw_team_id 
                else int(1 - row['HOME_TEAM_WINS']),
    axis=1
)

# 주요 시즌 필터링 (2012-2016)
key_seasons = [2012, 2013, 2014, 2015, 2016]
joined_filtered = joined[joined['SEASON'].isin(key_seasons)]

# 시즌별 3점슛 시도 횟수와 승률 통계
season_fg3a_stats = joined_filtered.groupby('SEASON').agg({
    'FG3A': 'mean',  # 경기당 평균 3점슛 시도
    'WIN': 'mean'    # 승률
}).reset_index()

season_fg3a_stats.columns = ['SEASON', 'FG3A_AVG', 'WIN_PCT']
season_fg3a_stats

In [ ]:
# 시각화 1: 시즌별 3점슛 시도 횟수와 승률 비교 (이중 Y축)
fig, ax1 = plt.subplots(figsize=(14, 7))

# 첫 번째 y축: 승률
color1 = '#1f77b4'
ax1.set_xlabel('시즌', fontsize=13, fontweight='bold')
ax1.set_ylabel('승률', fontsize=13, fontweight='bold', color=color1)
ax1.plot(season_fg3a_stats['SEASON'], season_fg3a_stats['WIN_PCT'], 
         marker='o', linewidth=3, markersize=10, 
         color=color1, label='승률')
ax1.tick_params(axis='y', labelcolor=color1, labelsize=11)
ax1.grid(True, alpha=0.2)

# 두 번째 y축: 3점슛 시도 횟수
ax2 = ax1.twinx()
color2 = '#2ca02c'
ax2.set_ylabel('경기당 3점슛 시도 횟수', fontsize=13, fontweight='bold', color=color2)
ax2.plot(season_fg3a_stats['SEASON'], season_fg3a_stats['FG3A_AVG'], 
         marker='s', linewidth=3, markersize=10, 
         color=color2, label='3점슛 시도')
ax2.tick_params(axis='y', labelcolor=color2, labelsize=11)

# 2015 우승 시즌 강조
ax1.axvline(x=2015, color='red', linestyle=':', 
            linewidth=3, alpha=0.6, label='2015 우승')

# 제목
plt.title('Golden State Warriors: 3점슛 시도 횟수와 승률의 관계 (2012-2016)', 
          fontsize=15, fontweight='bold', pad=20)

# 범례
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, 
           loc='upper left', fontsize=11, framealpha=0.9)

plt.tight_layout()
plt.show()

In [ ]:
# 시각화 2: 3점슛 시도 횟수 vs 승률 산점도 (상관관계 분석)
plt.figure(figsize=(10, 8))

# 산점도
plt.scatter(season_fg3a_stats['FG3A_AVG'], season_fg3a_stats['WIN_PCT'], 
            s=200, alpha=0.6, c=season_fg3a_stats['SEASON'], 
            cmap='viridis', edgecolors='black', linewidth=2)

# 각 점에 시즌 라벨 추가
for idx, row in season_fg3a_stats.iterrows():
    plt.annotate(f"{int(row['SEASON'])}", 
                 (row['FG3A_AVG'], row['WIN_PCT']),
                 fontsize=11, fontweight='bold',
                 ha='center', va='center')

# 추세선 추가
z = np.polyfit(season_fg3a_stats['FG3A_AVG'], season_fg3a_stats['WIN_PCT'], 1)
p = np.poly1d(z)
x_line = np.linspace(season_fg3a_stats['FG3A_AVG'].min(), season_fg3a_stats['FG3A_AVG'].max(), 100)
plt.plot(x_line, p(x_line), "r--", alpha=0.8, linewidth=2, label='추세선')

plt.xlabel('경기당 3점슛 시도 횟수', fontsize=13, fontweight='bold')
plt.ylabel('승률', fontsize=13, fontweight='bold')
plt.title('Golden State Warriors: 3점슛 시도 횟수 vs 승률 상관관계', 
          fontsize=15, fontweight='bold')
plt.colorbar(label='시즌')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# 상관계수 출력
correlation = season_fg3a_stats['FG3A_AVG'].corr(season_fg3a_stats['WIN_PCT'])
print(f"\n📊 3점슛 시도 횟수 vs 승률 상관계수: {correlation:.3f}")
if correlation > 0.7:
    print("→ 강한 양의 상관관계! 3점슛을 많이 시도할수록 승률이 높다!")
elif correlation > 0.4:
    print("→ 중간 정도의 양의 상관관계")
elif correlation > 0:
    print("→ 약한 양의 상관관계")
else:
    print("→ 상관관계가 없거나 음의 상관관계")